In [ ]:
import json
from elasticsearch import Elasticsearch
from credentials import auth
from elasticsearch.helpers import scan
from elasticsearch_dsl import Search, Q

In [ ]:
u,p=auth()
hosts ="https://ctesm1.discoverdollar.com:41532/"
es = Elasticsearch(hosts,  timeout=60000, use_ssl=True, verify_certs=False, 
ssl_show_warn=False, raise_on_error=False,http_auth=(u,p))

In [ ]:
index="cantire-2020-06"
body={"size":1000,
      "query":{
    "match_phrase":{
    "fileType": "Message/Email File"
    }}
}
res=es.search(index=index,body=body)

In [1]:
import re
import json
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_trf")


REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|_\-\–<>;:]')
emails=re.compile('[\w.+-]+@[\w-]+\.[\w.-]+')
anchor=re.compile("<[^ ]*>")
cid=re.compile("cid[^ ]*")
sqbrac=re.compile('\[[^ ]*\]')
has=re.compile("#[0-9a-bA-B]*")
def prepare_text(text):
    #text = text.lower()
    text=anchor.sub(" ",text)
    text=sqbrac.sub(" ",text)
    text=has.sub(" ",text)
    text=cid.sub(" ",text)
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text=" ".join(text.split())
    return text
def get_35(t):
    k=t.split()
    if len(k)<=35:
        return t
    else:
        return " ".join(k[len(k)-35:])

def get_name(l):
    if len(l)>1:
        l=l[1:]
    names=[]
    for i in l:
        r=re.search(r'[\w.+-]+@[\w-]+\.[\w.-]+',i if len(i)<100 else i[:100])
        if r:
            name=i[:r.start()]
            REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|\-\–<>;:]')
            name = REPLACE_BY_SPACE_RE.sub(' ', name)
            names.append(name)
            
        else:
            names.append("no-info")

In [ ]:
for i in range(1000):
        d=dict()
        content=res['hits']['hits'][i]['_source']['content']
        content.encode("ascii", "replace")
        text=" ".join(content.split())
        l=re.split("From:",text)
        l=[i.strip() for i in l]
        names=get_names(l)
        l=[prepare_text(i) for i in l]
        l=[get_35(i) for i in l]
    
        for i in len(l):
            s=l[i]
            d=dict()
            person=[]
            org=[]
            doc=nlp(s)
            num=[]
            for ent in doc.ents:
                if ent.label_=="PERSON":
                    person.append((ent.start_char, ent.end_char))
                elif ent.label_=="ORG":
                    org.append((ent.start_char, ent.end_char))
               
                
            if person:
                for j in range(len(org)):
                    if org[j][0]>person[-1][1]:
                        org=org[j:]
                        break
                else:
                    org=[]
               
                cardinals=re.findall('\d{3}[\. ]?\d{3}[\. ]?\d{4}|\d{2} \d{8}',s[person[-1][0]:])
                if cardinals:
                    ind=s.index(cardinals[0])
                    d['Name']=s[person[-1][0]:person[-1][1]+1]
                    if org:
                        d['Designation']=s[person[-1][1]+1:org[0][0]]
                        d['orgs']=s[org[0][0]:ind]
                    else:
                        d['org']=s[person[-1][0]:ind]
                    d['Contacts']=cardinals
            
            if d :           
                print(d)
            else:
                print(names[i])